In [ ]:
# !pip install chart_studio
# !pip install cufflinks
# !pip install plotly
# !pip install missingno

In [ ]:

import numpy     as np
import pandas    as pd

# Options for pandas
pd.options.display.max_columns = 30

import matplotlib.pyplot as plt
import seaborn   as sns
import pylab
import calendar
from   scipy import stats as sci
import missingno as msno
from   datetime import datetime

# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs   as go
from   plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Set global theme
cufflinks.set_config_file(world_readable=True, theme="pearl")

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import plotly.io as pio
pio.renderers.default = "colab"

# Warnings
import warnings 
%matplotlib inline
warnings.filterwarnings('ignore')

# Drive connect to get the data.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



# Data Loading exported From RandomForest algorthm's Notebook
loading one data set for october sales and the other for november then combining Them


In [ ]:
data_RF = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataPredicted_RandomForest.csv')
data_RF

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
...,...,...,...,...,...,...,...,...,...
8738115,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,NaN,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e
8738116,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,NaN,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e
8738117,2019-11-30 23:59:37 UTC,view,5699730,1487580004882580302,NaN,estel,2.70,422196217,dd6d8240-0896-4965-9344-110648581a51
8738118,2019-11-30 23:59:46 UTC,view,5830317,1487580009496313889,NaN,ellips,4.76,457678989,ee50b160-a4db-4722-8751-6812c5b38295


In [ ]:
data = data_RF.copy()


After concatenate data now we have 8738120 instance 

In [ ]:
df = data.copy()

# DATA EXPLORATION & MANUPILATION

In [ ]:
# df = data.copy().sample(n = 1000000)
df = data.copy()
# 'count_user' : how many times this user made an action on the site based on -partitioned by- the 'event_type'
df['count_user'] = df.groupby(['user_id', 'event_type'])['user_id'].transform('count')

# 'count_brand': how many times an action has been taken on this brand products based on the 'event_type'
df['count_brand'] = df.groupby(['brand', 'event_type'])['brand'].transform('count')

# 'count_event_type : how many times this event time has happened.
df['count_event_tye'] = df.groupby('event_type')['event_type'].transform('count')

# 'count_product' : how many times an action has been taken on this product based on the 'event_type'
df['count_product'] = df.groupby(['product_id', 'event_type'])['product_id'].transform('count')

# 'count-session' : Number of sessions do each user have.
df['count_session'] = df.groupby('user_session')['user_session'].transform('count')

# 'count_sessions_type' : Number of sessions do each user have per each 'event_type'
df['count_sessions_type'] = df.groupby(['user_session', 'event_type'])['product_id'].transform('count')

df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,count_user,count_brand,count_event_tye,count_product,count_session,count_sessions_type
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,58,27.0,7.0
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,46,27.0,7.0
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,95,55.0,5.0
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,767,27.0,7.0
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,482,55.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8738115,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,NaN,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0
8738116,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,NaN,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0
8738117,2019-11-30 23:59:37 UTC,view,5699730,1487580004882580302,NaN,estel,2.70,422196217,dd6d8240-0896-4965-9344-110648581a51,20,119933,3938296,191,1.0,1.0
8738118,2019-11-30 23:59:46 UTC,view,5830317,1487580009496313889,NaN,ellips,4.76,457678989,ee50b160-a4db-4722-8751-6812c5b38295,5,2862,3938296,1469,2.0,2.0


####Getting the size of the dataset.

In [ ]:
df.shape 

(8738120, 15)

###describe our data through some statistic methods

In [ ]:
df.describe()

,product_id,category_id,price,user_id,count_user,count_brand,count_event_tye,count_product,count_session,count_sessions_type
count,8.738120e+06,8.738120e+06,8.738120e+06,8.738120e+06,8.738120e+06,8.738120e+06,8.738120e+06,8.738120e+06,8.736670e+06,8.736670e+06
mean,5.475330e+06,1.547985e+18,8.315165e+00,5.042412e+08,1.160473e+02,1.021762e+05,2.878619e+06,7.951033e+02,4.969375e+01,2.579578e+01
std,1.313266e+06,1.606052e+17,1.895403e+01,8.354095e+07,4.316695e+02,1.111909e+05,1.076888e+06,2.196348e+03,1.155199e+02,7.924155e+01
min,3.752000e+03,1.487580e+18,-7.937000e+01,1.120748e+06,1.000000e+00,1.000000e+00,5.680410e+05,1.000000e+00,1.000000e+00,1.000000e+00
25%,5.723503e+06,1.487580e+18,2.050000e+00,4.694674e+08,1.000000e+01,1.834300e+04,1.687591e+06,6.400000e+01,5.000000e+00,3.000000e+00
50%,5.809910e+06,1.487580e+18,3.970000e+00,5.405799e+08,3.400000e+01,5.343000e+04,2.544192e+06,1.870000e+02,2.000000e+01,9.000000e+00
75%,5.854158e+06,1.487580e+18,6.780000e+00,5.611356e+08,9.700000e+01,1.699080e+05,3.938296e+06,5.820000e+02,5.300000e+01,2.400000e+01
max,5.909246e+06,2.195085e+18,3.277800e+02,5.799699e+08,9.402000e+03,3.492000e+05,3.938296e+06,2.728700e+04,2.854000e+03,2.605000e+03


###know some more info about our data like data types & memory usage

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8738120 entries, 0 to 8738119
Data columns (total 15 columns):
 #   Column               Dtype  
---  ------               -----  
 0   event_time           object 
 1   event_type           object 
 2   product_id           int64  
 3   category_id          int64  
 4   category_code        object 
 5   brand                object 
 6   price                float64
 7   user_id              int64  
 8   user_session         object 
 9   count_user           int64  
 10  count_brand          int64  
 11  count_event_tye      int64  
 12  count_product        int64  
 13  count_session        float64
 14  count_sessions_type  float64
dtypes: float64(3), int64(7), object(5)
memory usage: 1000.0+ MB


As we can see from above info() method, all the datatypes are correctly available. Hours is available as int64. Since hours is only used for grouping and other data analysis related works, its okay to have it in int64 format. Lets quickly check the minimum, average, maximum and other statics related to the numerical columns.

In [ ]:
df['brand'].value_counts()

irisk           935913
runail          864025
masura          642970
grattol         347942
ingarden        338591
                 ...  
invisibobble        10
voesh                8
dessata              4
shifei               4
vl-gel               3
Name: brand, Length: 244, dtype: int64

In [ ]:
df['category_code'].value_counts()

appliances.environment.vacuum             59761
stationery.cartrige                       26722
apparel.glove                             18232
furniture.living_room.cabinet             13439
accessories.bag                           11681
furniture.bathroom.bath                    9857
appliances.personal.hair_cutter            1643
accessories.cosmetic_bag                   1248
appliances.environment.air_conditioner      332
furniture.living_room.chair                 308
sport.diving                                  2
Name: category_code, dtype: int64

In [ ]:
# dropping dupplicates 
df.drop_duplicates()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,count_user,count_brand,count_event_tye,count_product,count_session,count_sessions_type
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,58,27.0,7.0
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,46,27.0,7.0
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,95,55.0,5.0
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,767,27.0,7.0
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,482,55.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8738114,2019-11-30 23:59:13 UTC,view,5629988,1487580009311764506,NaN,ingarden,1.19,579966747,1512be50-d0fd-4a92-bcd8-3ea3943f2a3b,4,130693,3938296,622,18.0,4.0
8738115,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,NaN,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0
8738117,2019-11-30 23:59:37 UTC,view,5699730,1487580004882580302,NaN,estel,2.70,422196217,dd6d8240-0896-4965-9344-110648581a51,20,119933,3938296,191,1.0,1.0
8738118,2019-11-30 23:59:46 UTC,view,5830317,1487580009496313889,NaN,ellips,4.76,457678989,ee50b160-a4db-4722-8751-6812c5b38295,5,2862,3938296,1469,2.0,2.0


## Feature Engineering for the 'event_time' colum

Event_time column has string "UTC" in it. So split the actual datetime and UTC seperate. Then convert the Event_time column into datetime datatype. Also, create new columns called date,time,hour,weekday,weeknum from the "event_time" column. Lets also change the weekday format from number to string like mon,tues,etc..

In [ ]:
# 1- splitting the "Datetime" column into "date", "hour" , "year" columns
df["date"] = df.event_time.apply(lambda dateNum : dateNum.split()[0])
df["hour"] = df.event_time.apply(lambda hourNum : hourNum.split()[1].split(":")[0])
df["year"] = df.event_time.apply(lambda yearNum : yearNum.split()[0].split("-")[0])

# 2- Getting the "weekday", "month" columns out of "date" column
df["weekday"] = df.date.apply(lambda dateStr : calendar.day_name[datetime.strptime(dateStr,"%Y-%m-%d").weekday()])
df["month"] = df.date.apply(lambda dateStr : calendar.month_name[datetime.strptime(dateStr,"%Y-%m-%d").month])


### filling category code missings with other category

In [ ]:
df.loc[df['category_code'].isnull(),'category_code'] = 'other'
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,count_user,count_brand,count_event_tye,count_product,count_session,count_sessions_type,date,hour,year,weekday,month
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,58,27.0,7.0,2019-10-01,00,2019,Tuesday,October
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,46,27.0,7.0,2019-10-01,00,2019,Tuesday,October
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,other,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,95,55.0,5.0,2019-10-01,00,2019,Tuesday,October
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,767,27.0,7.0,2019-10-01,00,2019,Tuesday,October
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,other,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,482,55.0,5.0,2019-10-01,00,2019,Tuesday,October
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8738115,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,other,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0,2019-11-30,23,2019,Saturday,November
8738116,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,other,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0,2019-11-30,23,2019,Saturday,November
8738117,2019-11-30 23:59:37 UTC,view,5699730,1487580004882580302,other,estel,2.70,422196217,dd6d8240-0896-4965-9344-110648581a51,20,119933,3938296,191,1.0,1.0,2019-11-30,23,2019,Saturday,November
8738118,2019-11-30 23:59:46 UTC,view,5830317,1487580009496313889,other,ellips,4.76,457678989,ee50b160-a4db-4722-8751-6812c5b38295,5,2862,3938296,1469,2.0,2.0,2019-11-30,23,2019,Saturday,November


### 'Event_type' Encoding Using ONE HOT ENCODING

In [ ]:
event_type_enc = pd.get_dummies(df['event_type'])
event_type_enc

,cart,purchase,remove_from_cart,view
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
8738115,0,0,1,0
8738116,0,0,1,0
8738117,0,0,0,1
8738118,0,0,0,1


#### Adding the encoded columns into the original dataframe

In [ ]:
df = pd.concat([df, event_type_enc], axis = 1)
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,count_user,count_brand,count_event_tye,count_product,count_session,count_sessions_type,date,hour,year,weekday,month,cart,purchase,remove_from_cart,view
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,58,27.0,7.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,46,27.0,7.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,other,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,95,55.0,5.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,767,27.0,7.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,other,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,482,55.0,5.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8738115,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,other,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0,2019-11-30,23,2019,Saturday,November,0,0,1,0
8738116,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,other,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0,2019-11-30,23,2019,Saturday,November,0,0,1,0
8738117,2019-11-30 23:59:37 UTC,view,5699730,1487580004882580302,other,estel,2.70,422196217,dd6d8240-0896-4965-9344-110648581a51,20,119933,3938296,191,1.0,1.0,2019-11-30,23,2019,Saturday,November,0,0,0,1
8738118,2019-11-30 23:59:46 UTC,view,5830317,1487580009496313889,other,ellips,4.76,457678989,ee50b160-a4db-4722-8751-6812c5b38295,5,2862,3938296,1469,2.0,2.0,2019-11-30,23,2019,Saturday,November,0,0,0,1


### 'purchase' column ONE HOT ENCODING

In [ ]:
purchase_enc = pd.get_dummies(df['purchase'])
purchase_enc

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
8738115,1,0
8738116,1,0
8738117,1,0
8738118,1,0


#### Renaming encoded columns

In [ ]:
purchase_enc.rename(columns = {0:'no_purchase', 1 : 'yes_purchase'}, inplace = True)
purchase_enc

,no_purchase,yes_purchase
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
8738115,1,0
8738116,1,0
8738117,1,0
8738118,1,0


#### Adding purchase status columns into the df

In [ ]:
df = pd.concat([df, purchase_enc], axis = 1)
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,count_user,count_brand,count_event_tye,count_product,count_session,count_sessions_type,date,hour,year,weekday,month,cart,purchase,remove_from_cart,view,no_purchase,yes_purchase
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,58,27.0,7.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0,1,0
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,46,27.0,7.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0,1,0
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,other,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,95,55.0,5.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0,1,0
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,277330,2544192,767,27.0,7.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0,1,0
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,other,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,24299,2544192,482,55.0,5.0,2019-10-01,00,2019,Tuesday,October,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8738115,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,other,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0,2019-11-30,23,2019,Saturday,November,0,0,1,0,1,0
8738116,2019-11-30 23:59:24 UTC,remove_from_cart,5608703,1487580005553668971,other,matrix,9.52,566513990,b5053009-b3c4-4581-b05f-6199d1a2037e,7,4425,1687591,651,10.0,6.0,2019-11-30,23,2019,Saturday,November,0,0,1,0,1,0
8738117,2019-11-30 23:59:37 UTC,view,5699730,1487580004882580302,other,estel,2.70,422196217,dd6d8240-0896-4965-9344-110648581a51,20,119933,3938296,191,1.0,1.0,2019-11-30,23,2019,Saturday,November,0,0,0,1,1,0
8738118,2019-11-30 23:59:46 UTC,view,5830317,1487580009496313889,other,ellips,4.76,457678989,ee50b160-a4db-4722-8751-6812c5b38295,5,2862,3938296,1469,2.0,2.0,2019-11-30,23,2019,Saturday,November,0,0,0,1,1,0


### 'category_code' Encoding Using ONE HOT ENCODING

In [ ]:
category_code_enc = pd.get_dummies(df['category_code'])
category_code_enc

,accessories.bag,accessories.cosmetic_bag,apparel.glove,appliances.environment.air_conditioner,appliances.environment.vacuum,appliances.personal.hair_cutter,furniture.bathroom.bath,furniture.living_room.cabinet,furniture.living_room.chair,other,sport.diving,stationery.cartrige
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8738115,0,0,0,0,0,0,0,0,0,1,0,0
8738116,0,0,0,0,0,0,0,0,0,1,0,0
8738117,0,0,0,0,0,0,0,0,0,1,0,0
8738118,0,0,0,0,0,0,0,0,0,1,0,0


#### Adding the encoded columns into the original dataframe

In [ ]:
df = pd.concat([df, category_code_enc], axis = 1)
df

## Exporting the Manipulated data into file

now we have day, time , hour , year month and date

In [ ]:
# df.to_csv('data_preprocessed.csv', index = False)